In [7]:
import pandas as pd
import sys

In [8]:
tbl = pd.read_csv("test.smbtbl", delimiter='|')
tbl.set_index("MLo", inplace=True)
print(tbl)
print(tbl.columns)

     Type  Lenght     Name   Scope LineNumbers
MLo                                           
3     int       0        x    main      26 30 
5     int       0    input  global         30 
2    void       0     main  global         25 
0     int       0   funcao  global      19 29 
4     int       0  bababoi    main      27 28 
1     int       0  bababoi  funcao   20 21 22 
Index(['Type', 'Lenght', 'Name', 'Scope', 'LineNumbers'], dtype='object')


In [9]:
file_path = 'test.quad'
commands = []
with open(file_path, 'r') as file:
    for line in file:
        quad = line.strip('(')
        quads = quad[:-2].split(', ')
        commands.append(quads)
print(commands[0][3])

-


In [10]:

usedIndexes = []
def fixIndex(line):
    global usedIndexes
    tobeUsed = []
    for command in commands[line + 1:]:
        # index = command[1].strip("$t")
        for i in range(1,4):
            if "$t" in command[i]:
                #print(command[i])
                if command[i][2:] in usedIndexes and command[i][2:] not in tobeUsed:
                    tobeUsed.append(command[i][2:])
    print(usedIndexes, tobeUsed)
    usedIndexes = list(set(usedIndexes) - set(set(usedIndexes) - set(tobeUsed)))

# tobeUsed

In [11]:
tbl.index[tbl["Name"] == "main"][0]

2

In [12]:
#addi beq bne bgt blt lw sw j jal (jar) add sub div mult and or
stack = []
stacksize = 0
stackpointer = 0
assembly = open("test.asm", 'w')
assembly.write("j "+str(tbl.index[tbl["Name"] == "main"][0]))
for i, command in enumerate(commands):
    for reg in command[1:]:
        if "$t" in reg and reg[2:] not in usedIndexes:
            usedIndexes.append(reg[2:])
    match command[0]:
    #     case 'FUNC':
    #         #print("eh func")
    #     case 'ARG':
    #         print("eh arg")
    #     case 'LOAD':
    #         print("eh load")
    #     case 'IFF':
    #         print("eh IFF")
    #     case 'GOTO':
    #         print("eh GOTO")
    #     case 'LAB':
    #         print("eh LAB")
    #     case 'WHILE':
    #         print("eh WHILE")
    #     case 'RET':
    #         print("eh RET")
    #     case 'END':
    #         print("eh END")
    #     case 'CALL':
    #         print("eh CALL")
    #     case 'ALLOC':
    #         print("eh ALLOC")
    #     case 'STORE':
    #         print("eh STORE")
    #     case 'PARAM':
    #         print("eh PARAM")
    #     case 'SOM':
    #         print("eh SOM")
    #     case 'SUB':
    #         print("eh func")
         case 'MULT':
            assembly.write("mult "+command[1]+" "+command[2]+" "+command[3]+"\n")
    #     case 'LT':
    #         print("eh LT")
    #     case 'LET':
    #         print("eh LET")
    #     case 'GT':
    #         print("eh GT")
    #     case 'GET':
    #         print("eh GET")
    #     case 'COMP':
    #         print("eh COMP")
    #     case 'NEQ':
    #         print("eh NEQ")
    #     case 'EQ':
    #         print("eh EQ")
    fixIndex(i)
#print(usedIndexes)
assembly.close()

[] []
[] []
['0'] ['0']
['0'] []
['1'] ['1']
['1'] []
[] []
[] []
[] []
[] []
['2'] ['2']
['2'] []
['3'] []
['4'] ['4']
['4'] []
[] []
